In [7]:
import numpy as np
import cv2
import os

In [8]:
#RNN

def dist(x, y):
    return np.sqrt(sum((x-y)**2))

def rnn(x , y, q_point, k=5):
    
    vals = []
    m = x.shape[0]
    for i in range(m):
        d = dist(q_point, x[i])
        vals.append((d, y[i]))
        
    vals = sorted(vals)
    vals = vals[:k]
    
    vals = np.array(vals)
    new_vals = np.unique(vals[:1], return_counts = True)
    
    index = new_vals[1].argmax() # argmax() returns the indices of the max value in an array  
    pred = new_vals[0][index]
    
    return pred

In [9]:
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

skip = 0
dataset_path = 'C:/Users/samee/Desktop/Data Science/ML_Coding_Blocks/openCV_Face_Recognition/'

face_data = []
labels = []

In [10]:
class_id = 0 # labels for the given file
names = {} # mapping btw id and name

# data preparation

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        data_item = np.loadtxt(dataset_path +fx) 
        face_data.append(data_item)
        
        # creating labels for a class 
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_data = np.concatenate(face_data, axis=0)
face_labels= np.concatenate(labels, axis=0)
        
print(face_data.shape)
print(face_labels.shape)

(87, 30000)
(87,)


In [11]:
#trainset = np.concatenate((face_data, face_labels), axis = 1)

In [12]:
while True:
    ret, frame = cap.read()
    if ret == False:
        continue
        
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    
    for face in faces:
        x, y, w, h = face
        
        offset = 10
        face_section = frame[y-offset:y+h+offset, x-offset:y+w+offset]
        face_section = cv2.resize(face_section, (100,100))
        
        out = rnn(face_data, face_labels, face_section.flatten())
        
        pred_name = names[int(out)]
        cv2.putText(frame, pred_name, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 255), 2)
        
    cv2.imshow('faces', frame)
        
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()    
        